# Basic Usage

Aim: Use the Controller to recover a simple ground truth theory from noisy data.

In [ ]:
import numpy as np
from autora.experimentalist.pipeline import make_pipeline
from autora.variable import VariableCollection, Variable
from sklearn.linear_model import LinearRegression
from autora_workflow import Cycle
from itertools import takewhile

In [ ]:
def ground_truth(x):
    return x + 1

The space of allowed x values is the integers between 0 and 10 inclusive, and we record the allowed output values as well.

In [ ]:
metadata_0 = VariableCollection(
   independent_variables=[Variable(name="x1", allowed_values=range(11))],
   dependent_variables=[Variable(name="y", value_range=(-20, 20))],
   )

The experimentalist is used to propose experiments.
Since the space of values is so restricted, we can just sample them all each time.

In [ ]:
example_experimentalist = make_pipeline(
    [metadata_0.independent_variables[0].allowed_values])

When we run a synthetic experiment, we get a reproducible noisy result:

In [ ]:
def get_example_synthetic_experiment_runner():
    rng = np.random.default_rng(seed=180)
    def runner(x):
        return ground_truth(x) + rng.normal(0, 0.1, x.shape)
    return runner
example_synthetic_experiment_runner = get_example_synthetic_experiment_runner()
example_synthetic_experiment_runner(np.array([1]))

The theorist "tries" to work out the best theory. We use a trivial scikit-learn regressor.

In [ ]:
example_theorist = LinearRegression()

    We initialize the Controller with the metadata describing the domain of the theory,
    the theorist, experimentalist and experiment runner,
    as well as a monitor which will let us know which cycle we're currently on.

In [ ]:
cycle = Cycle(
    metadata=metadata_0,
    theorist=example_theorist,
    experimentalist=example_experimentalist,
    experiment_runner=example_synthetic_experiment_runner,
    monitor=lambda state: print(f"Generated {len(state.theories)} theories"),
)
cycle # doctest: +ELLIPSIS

We can run the cycle by calling the run method:

In [ ]:
cycle.run(num_cycles=3)  # doctest: +ELLIPSIS

We can now interrogate the results. The first set of conditions which went into the
experiment runner were:

In [ ]:
cycle.data.conditions[0]

The observations include the conditions and the results:

In [ ]:
cycle.data.observations[0]

In the third cycle (index = 2) the first and last values are different again:

In [ ]:
cycle.data.observations[2][[0,-1]]

The best fit theory after the first cycle is:

In [ ]:
cycle.data.theories[0]

In [ ]:
def report_linear_fit(m: LinearRegression,  precision=4):
    s = f"y = {np.round(m.coef_[0].item(), precision)} x " \
        f"+ {np.round(m.intercept_.item(), 4)}"
    return s
report_linear_fit(cycle.data.theories[0])

The best fit theory after all the cycles, including all the data, is:

In [ ]:
report_linear_fit(cycle.data.theories[-1])

This is close to the ground truth theory of x -> (x + 1)
We can also run the cycle with more control over the execution flow:

In [ ]:
next(cycle) # doctest: +ELLIPSIS

next(cycle) # doctest: +ELLIPSIS

In [ ]:
next(cycle) # doctest: +ELLIPSIS

We can continue to run the cycle as long as we like,
with a simple arbitrary stopping condition like the number of theories generated:

In [ ]:
_ = list(takewhile(lambda c: len(c.data.theories) < 9, cycle))

... or the precision (here we keep iterating while the difference between the gradients
of the second-last and last cycle is larger than 1x10^-3).

In [ ]:
_ = list(
        takewhile(
            lambda c: np.abs(c.data.theories[-1].coef_.item() -
                           c.data.theories[-2].coef_.item()) > 1e-3,
            cycle
        )
    )


... or continue to run as long as we like:

In [ ]:
_ = cycle.run(num_cycles=100) # doctest: +ELLIPSIS
